1. Parsing the System of Equations

In [1]:
def parse_equations(file_path):
    A = [] 
    B = []  

    with open(file_path, 'r') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                lhs, rhs = line.strip().split('=')
            except ValueError:
                print(f"Error parsing line {line_number}: '{line.strip()}'. Ensure it contains exactly one '=' sign.")
                continue  

            rhs = rhs.strip()
            try:
                B.append(float(rhs))
            except ValueError:
                print(f"Error converting RHS to float on line {line_number}: '{rhs}'.")
                continue  

            
            coeffs = {'x': 0.0, 'y': 0.0, 'z': 0.0}

            lhs_modified = lhs.replace('-', '+-')
            terms = lhs_modified.split('+')

            for term in terms:
                term = term.strip()  
                if not term:
                    continue  

                if 'x' in term:
                    var = 'x'
                elif 'y' in term:
                    var = 'y'
                elif 'z' in term:
                    var = 'z'
                else:
                    print(f"Warning: Unrecognized variable in term '{term}' on line {line_number}. Skipping term.")
                    continue 

                coeff_str = term.replace(var, '').strip()

                # print(f"Processing term '{term}': extracted coefficient string '{coeff_str}'")

                if coeff_str in ['', '+']:
                    coefficient = 1.0
                elif coeff_str == '-':
                    coefficient = -1.0
                else:
                    try:
                        coefficient = float(coeff_str)
                    except ValueError:
                        print(f"Error converting coefficient to float in term '{term}' on line {line_number}.")
                        coefficient = 0.0  

                coeffs[var] = coefficient

            A.append([coeffs['x'], coeffs['y'], coeffs['z']])

    return A, B

if __name__ == "__main__":
    file_path = 'equations.txt'
    A, B = parse_equations(file_path)

    print("Matrix A:")
    for row in A:
        print(row)

    print("\nVector B:")
    print(B)


Matrix A:
[2.0, 3.0, -1.0]
[1.0, -1.0, 4.0]
[3.0, 1.0, 2.0]

Vector B:
[5.0, 6.0, 7.0]


2. Matrix and Vector Operations

2.1. Determinant:

In [2]:
def determinant_3X3(matrix):
    if len(matrix) != 3 or any(len(row) != 3 for row in matrix):
        raise ValueError("Matrix must be 3x3.")
    
    a11, a12, a13 = matrix[0]
    a21, a22, a23 = matrix[1]
    a31, a32, a33 = matrix[2]
    
    det = (a11 * (a22 * a33 - a23 * a32)-
           a12 *(a21 * a33 - a23 * a31)+
           a13 *(a21 * a32 - a22 * a31))
    return det


print(determinant_3X3(A))

14.0


2.2. Trace

In [3]:
def trace(matrix):
    return matrix[0][0] + matrix[1][1] + matrix[2][2]


print(trace(A))

3.0


2.3. Vector norm

In [4]:
def vector_norm(vector):
    if len(vector) != 3:
         raise ValueError("Vector must be of length 3.")
    return (vector[0]**2 + vector[1]**2 + vector[2]**2)**0.5

print(vector_norm(B))

10.488088481701515


2.4. Transpose of matrix

In [5]:
def transpose_of_matrix(matrix):
    if len(matrix) != 3 or any(len(row) != 3 for row in matrix):
        raise ValueError("Matrix must be 3x3.")
    
    num_rows = len(matrix)
    num_cols = len(matrix[0])
    
    transposed_matrix = []
    for i in range(num_rows):
        transposed_row = []
        for j in range(num_cols):
            transposed_row.append(matrix[j][i])
        transposed_matrix.append(transposed_row)
    
    return transposed_matrix


transposed_A = transpose_of_matrix(A)
for row in transposed_A:
    print(row)

[2.0, 1.0, 3.0]
[3.0, -1.0, 1.0]
[-1.0, 4.0, 2.0]


2.5. Matrix-vector multiplication

In [6]:
def matrix_vector_multiplication(matrix, vector):
    num_rows = len(matrix)
    num_cols = len(matrix[0])
    if num_cols != len(vector):
        raise ValueError("Number of columns must be equal to the number of elements in vector.")
    result = []
    for i in range(num_rows):
        sum = 0.0
        for j in range(num_cols):
            sum += matrix[i][j] * vector[j]
        result.append(sum)
    return result


print(matrix_vector_multiplication(A, B))
        

[21.0, 27.0, 35.0]


3. Solving using Cramer’s Rule

In [10]:
def replace_column(matrix, vector, column):
    new_matrix = []
    for i in range(len(matrix)):
        new_row = matrix[i].copy()
        new_row[column] = vector[i]
        new_matrix.append(new_row)
    return new_matrix

def solve_cramers_rule(matrix,vector):
    determinant = determinant_3X3(matrix)
    if determinant == 0:
        raise ValueError("The system has no unique solution (determinant is zero).")
    matrix_x =replace_column(matrix, vector, 0)
    matrix_y = replace_column(matrix, vector, 1)
    matrix_z = replace_column(matrix, vector, 2)
    
    det_matrix_x = determinant_3X3(matrix_x)
    det_matrix_y = determinant_3X3(matrix_y)
    det_matrix_z = determinant_3X3(matrix_z)
    
    x = det_matrix_x / determinant
    y = det_matrix_y / determinant
    z = det_matrix_z / determinant
    
    return {'x': x, 'y': y, 'z': z}

solutions = solve_cramers_rule(A, B)
print("\nSolutions using Cramer's Rule:")
print(f"x = {solutions['x']}")
print(f"y = {solutions['y']}")
print(f"z = {solutions['z']}")



Solutions using Cramer's Rule:
x = 0.35714285714285715
y = 2.0714285714285716
z = 1.9285714285714286


4. Solving using Inversion

In [8]:
def get_minor(matrix, i, j):
    minor = [row[:j] + row[j+1:] for idx, row in enumerate(matrix) if idx!=i]
    return minor

def determinant_2x2(matrix):
    if len(matrix) != 2 or any(len(row) != 2 for row in matrix):
        raise ValueError("The minor matrix must be 2x2.")
    a, b = matrix[0]
    c, d = matrix[1]
    return a * d - b * c


def cofactor_matrix(matrix):
    cof_matrix = []
    for i in range(len(matrix)):
        cof_row =[]
        for j in range(len(matrix[0])):
            minor = get_minor(matrix, i, j)
            det_minor = determinant_2x2(minor)
            cof = ((-1) ** (i+j)) * det_minor
            cof_row.append(cof)
        cof_matrix.append(cof_row)
    return cof_matrix


def adjugate_matrix(matrix):
    cof_matrix = cofactor_matrix(matrix)
    adjugate_matrix = transpose_of_matrix(cof_matrix)
    return adjugate_matrix


def inverse_matrix(matrix):
    determinant = determinant_3X3(matrix)
    if determinant == 0:
        raise ValueError("Matrix is singular and cannot be inverted.")
    adjugate = adjugate_matrix(matrix)
    inverse=[]
    for row in adjugate:
        inverse_row = []
        for element in row:
            inverse_row.append(element/determinant)
        inverse.append(inverse_row)
    return inverse


def solve_using_inverse_matrix(matrix, vector):
    inverse = inverse_matrix(matrix)
    x = matrix_vector_multiplication(inverse, vector)
    return x
    


solutions_inverse = solve_using_inverse_matrix(A, B)
print("\nSolutions using Matrix Inversion:")
print(f"x = {solutions_inverse[0]}")
print(f"y = {solutions_inverse[1]}")
print(f"z = {solutions_inverse[2]}")


Solutions using Matrix Inversion:
x = 0.35714285714285765
y = 2.071428571428571
z = 1.9285714285714288
